<a href="https://colab.research.google.com/github/mishraprashant1/MachineLearning/blob/master/JantaHack_E_Commerce_Solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [133]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

import lightgbm as lgb
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

In [134]:
#from google.colab import files
#file = files.upload()

In [135]:
train = pd.read_csv("train_8wry4cB.csv")
test = pd.read_csv("test_Yix80N0.csv")

In [136]:
train = train.set_index(['session_id', 'startTime', 'endTime', 'gender']).stack().str.split(";",expand=True).stack().unstack(-2).reset_index(-1,drop=True).reset_index()

In [137]:
train.head(train.shape[0]-1)

,session_id,startTime,endTime,gender,ProductList
0,u10001,14/11/14 0:02,14/11/14 0:02,female,A00001/B00001/C00001/D00001/
1,u10002,12/12/14 14:12,12/12/14 14:12,female,A00002/B00002/C00002/D24897/
2,u10003,14/11/14 0:02,14/11/14 0:16,female,A00002/B00002/C00002/D00002/
3,u10003,14/11/14 0:02,14/11/14 0:16,female,A00002/B00002/C00003/D00003/
4,u10003,14/11/14 0:02,14/11/14 0:16,female,A00002/B00002/C00007/D00007/
...,...,...,...,...,...
23245,u24996,22/12/14 22:31,22/12/14 22:35,female,A00002/B00001/C00010/D36037/
23246,u24997,22/12/14 22:52,22/12/14 22:53,female,A00002/B00002/C00002/D15648/
23247,u24997,22/12/14 22:52,22/12/14 22:53,female,A00002/B00002/C00007/D01372/
23248,u24997,22/12/14 22:52,22/12/14 22:53,female,A00002/B00002/C00007/D36051/


In [138]:
train[['category', 'sub category', 'sub-sub category', 'product', '/']] = (pd.DataFrame(train['ProductList'].str.split('/').values.tolist()))

In [139]:
train.head()

,session_id,startTime,endTime,gender,ProductList,category,sub category,sub-sub category,product,/
0,u10001,14/11/14 0:02,14/11/14 0:02,female,A00001/B00001/C00001/D00001/,A00001,B00001,C00001,D00001,
1,u10002,12/12/14 14:12,12/12/14 14:12,female,A00002/B00002/C00002/D24897/,A00002,B00002,C00002,D24897,
2,u10003,14/11/14 0:02,14/11/14 0:16,female,A00002/B00002/C00002/D00002/,A00002,B00002,C00002,D00002,
3,u10003,14/11/14 0:02,14/11/14 0:16,female,A00002/B00002/C00003/D00003/,A00002,B00002,C00003,D00003,
4,u10003,14/11/14 0:02,14/11/14 0:16,female,A00002/B00002/C00007/D00007/,A00002,B00002,C00007,D00007,


In [140]:
train_v1=train
train_v1['SecsTimeDiff'] = np.abs(pd.to_datetime(train_v1['endTime'])-pd.to_datetime(train_v1['startTime'])).dt.total_seconds()
train_v1['MinsTimeDiff'] = np.abs(pd.to_datetime(train_v1['endTime'])-pd.to_datetime(train_v1['startTime'])).dt.total_seconds() / 60
train_v1.drop(['startTime', 'endTime', 'ProductList', '/', 'SecsTimeDiff'], axis=1, inplace=True)
train_v1 = train_v1[['session_id', 'category', 'sub category', 'sub-sub category', 'product', 'MinsTimeDiff', 'gender']]
le = LabelEncoder()
train_v1.gender = le.fit_transform(train_v1.gender)
train_v1.replace('([A-Za-z]+)', '', regex=True, inplace=True)
train_v1 = train_v1.apply(pd.to_numeric)
train_v1 = train_v1.sample(frac=1).reset_index(drop=True)
train_v1.head()

,session_id,category,sub category,sub-sub category,product,MinsTimeDiff,gender
0,17595,2,2,7,8480,0.0,0
1,23981,1,1,182,25978,15.0,1
2,21876,2,2,7,20210,7.0,0
3,21939,1,31,91,29634,2.0,1
4,17358,2,3,5,24750,17.0,0


In [147]:
Y = train_v1['gender']
X = train_v1.drop('gender',axis=1)
#X["session_id"].to_numeric()
#X = X.to_numeric()
#Y = Y.to_numeric()
#pd.to_numeric(X["session_id"])

In [148]:
x_train, x_test, y_train, y_test = train_test_split(X,Y,test_size=0.2)

In [149]:
param_grid = {
    'num_leaves': [25,30,35,40,50],
    'max_depth': [6,8,10,12,14,20],
    'learning_rate': [0.01,0.05,0.8,0.1,0.14],
    'num_leaves': [50, 100, 200],
    'max_bin': [150,200,225,250],
}

In [150]:
model = lgb.LGBMClassifier()

In [151]:
gs = GridSearchCV(
        estimator=model,
        param_grid=param_grid, 
        n_jobs=-1, 
        scoring = 'accuracy', 
        cv=5,
        verbose=2
    )
fitted_model = gs.fit(x_train, y_train)

Fitting 5 folds for each of 360 candidates, totalling 1800 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:    8.2s
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:   43.8s
[Parallel(n_jobs=-1)]: Done 361 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 644 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done 1009 tasks      | elapsed:  4.9min
[Parallel(n_jobs=-1)]: Done 1454 tasks      | elapsed:  7.0min
[Parallel(n_jobs=-1)]: Done 1800 out of 1800 | elapsed:  8.6min finished


In [154]:
fitted_model.score(x_test,y_test)

0.9438830359062568